In [19]:
# Importa le librerie
import math
import random 
import statistics
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import set_option
from numpy import mean
from numpy import std
from numpy import arange
from pandas import read_csv
from sklearn.feature_selection import VarianceThreshold
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
import tensorflow as tf
from keras.callbacks import Callback
from keras import backend
from keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from math import floor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from numpy import sqrt
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import PowerTransformer
from numpy.random import seed
#import talib
#rom talib.abstract import *
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

import datetime
from datetime import datetime, timezone, date, timedelta
import time

import tweepy

import coinbasepro as cbp
import requests

from textblob import TextBlob
import re
import string

from numpy import mean
from numpy import median

import csv

import json

from pytrends.request import TrendReq



In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [36]:
df = pd.read_csv('data/bitcoin.csv', index_col='time')

In [4]:
df.head()

,low,high,open,close,volume,greed_and_fear,google_trend,sentiment_medio,difficulty
time,,,,,,,,,
2022-12-11,17073.19,17271.92,17128.10,17085.21,9948.852670,26,30,0.057855,3.424433e+13
2022-12-10,17093.42,17227.64,17130.49,17128.10,7860.586876,27,28,0.023664,3.424433e+13
2022-12-09,17060.69,17352.62,17226.03,17130.59,20976.636999,26,28,0.079547,3.424433e+13
2022-12-08,16738.00,17300.59,16839.76,17226.01,23533.234537,25,28,0.098894,3.424433e+13
2022-12-07,16679.52,17140.22,17089.18,16840.00,22635.468488,29,28,0.098525,3.424433e+13


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, 2022-12-11 to 2022-06-11
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   low              184 non-null    float64
 1   high             184 non-null    float64
 2   open             184 non-null    float64
 3   close            184 non-null    float64
 4   volume           184 non-null    float64
 5   greed_and_fear   184 non-null    int64  
 6   google_trend     184 non-null    int64  
 7   sentiment_medio  184 non-null    float64
 8   difficulty       184 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 14.4+ KB


**test tensorflow**

In [6]:
msg = tf.constant('Hello, TensorFlow!')
tf.print(msg)

Hello, TensorFlow!


In [7]:
df.shape

(184, 9)

In [8]:
# Stampa la versione di TensorFlow in uso
print(tf.__version__)

2.9.0


**Data preparation**

In [9]:
# Usa il dataframe per ottenere gli array di input e output
X = df.drop(columns=['close']) # tutte le colonne tranne 'close' sono gli input
y = df['close'] # la colonna 'close' è l'output

# Divide il dataset in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Standardizza le features con MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**Model preparation and compilation**

In [10]:
# Costruisci il modello di deep learning
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=184, activation='relu', input_dim=(X_train.shape[1])))
model.add(tf.keras.layers.Dense(units=32, activation='relu'))
model.add(tf.keras.layers.Dense(units=1))


# Compila il modello
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])

**Model training**

In [11]:
# Addestra il modello sui dati di addestramento
model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0)


In [12]:
# Valuta il modello utilizzando il set di test
model.summary()

predict = model.predict(X_test, verbose=0)

metrics = model.evaluate(X_test, y_test, verbose=2)
mae = mean_absolute_error(y_test, predict)
rmse = sqrt(mean_squared_error(y_test, predict))
print('MAE before inverse Scaling: %f' % mae)
print('RMSE before inverse Scaling: %f' % rmse)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 184)               1656      
                                                                 
 dense_1 (Dense)             (None, 32)                5920      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 7,609
Trainable params: 7,609
Non-trainable params: 0
_________________________________________________________________
2/2 - 0s - loss: 4330822.0000 - root_mean_squared_error: 2081.0627 - 91ms/epoch - 45ms/step
MAE before inverse Scaling: 1572.194595
RMSE before inverse Scaling: 2081.062751


In [13]:
# Ottieni la data di oggi
today = datetime(2022, 12, 11).date()

# Aggiungi un giorno alla data di oggi
tomorrow = today + timedelta(days=1)

# Crea un dizionario vuoto con i dati di domani
data_tomorrow = {}

# Aggiungi la data di domani al dizionario
data_tomorrow['time'] = tomorrow

# Aggiungi le altre colonne vuote al dizionario
data_tomorrow['low'] = 0
data_tomorrow['high'] = 0
data_tomorrow['open'] = 0
data_tomorrow['close'] = 0
data_tomorrow['volume'] = 0
data_tomorrow['greed_and_fear'] = 0
data_tomorrow['google_trend'] = 0
data_tomorrow['sentiment_medio'] = 0
data_tomorrow['difficulty'] = 0

# Crea un dataframe con i dati di domani
df_tomorrow = pd.DataFrame(data_tomorrow, index=[0])

df = df.reset_index()

# Unisci il dataframe di domani al dataframe storico
df = pd.concat([df_tomorrow,df.loc[:]]).reset_index(drop=True)

df = df.set_index('time')

df = df.replace(0, np.nan)
df.head()

,low,high,open,close,volume,greed_and_fear,google_trend,sentiment_medio,difficulty
time,,,,,,,,,
2022-12-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-11,17073.19,17271.92,17128.10,17085.21,9948.852670,26.0,30.0,0.057855,3.424433e+13
2022-12-10,17093.42,17227.64,17130.49,17128.10,7860.586876,27.0,28.0,0.023664,3.424433e+13
2022-12-09,17060.69,17352.62,17226.03,17130.59,20976.636999,26.0,28.0,0.079547,3.424433e+13
2022-12-08,16738.00,17300.59,16839.76,17226.01,23533.234537,25.0,28.0,0.098894,3.424433e+13


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185 entries, 2022-12-12 to 2022-06-11
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   low              184 non-null    float64
 1   high             184 non-null    float64
 2   open             184 non-null    float64
 3   close            184 non-null    float64
 4   volume           184 non-null    float64
 5   greed_and_fear   184 non-null    float64
 6   google_trend     184 non-null    float64
 7   sentiment_medio  184 non-null    float64
 8   difficulty       184 non-null    float64
dtypes: float64(9)
memory usage: 14.5+ KB


In [15]:
X = df.drop(columns=['close']) # tutte le colonne tranne 'close' sono gli input

In [16]:
tomorrow = X[:1]

In [17]:
tomorrow.head()

,low,high,open,volume,greed_and_fear,google_trend,sentiment_medio,difficulty
time,,,,,,,,
2022-12-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Usa il modello addestrato per fare previsioni sui dati di domani
next_day_predictions = model.predict(tomorrow)

print(next_day_predictions)

1/1 [==============================] - 0s 39ms/step
[[23.77595]]


**Scarico i dati di oggi**

In [35]:
#prezzo bictoin
client = cbp.PublicClient()
candles = client.get_product_historic_rates(product_id="BTC-USD", start="2022-12-21T00:00:00", stop="2022-12-22T00:00:00", granularity=86400)
df_domani = pd.DataFrame(candles)

# Inizializza la lista data_list
data_list = []

# Effettua 31 chiamate all'API di Alternative.me
for i in range(2):
  # Prepara la richiesta HTTP
  url = "https://api.alternative.me/fng/?limit=204&timestamp=" + str(i)
  headers = {'Content-Type': 'application/json'}

  # Effettua la richiesta HTTP
  response = requests.get(url, headers=headers)

  # Carica i dati in formato JSON
  data = response.json()

  # Estrai il valore "value" e il timestamp
  value = data["data"][i]["value"]
  timestamp = data["data"][i]["timestamp"]

  # Aggiungi i valori "value" e "timestamp" nella lista data_list
  data_list.append((value, timestamp))

  # Per ogni tupla nella lista data_list
for value, timestamp in data_list:
  # Converti il timestamp in una data
  date = datetime.fromtimestamp(int(timestamp)).strftime('%Y-%m-%d')
  #date = pd.to_datetime(date)
  # Seleziona la riga del dataframe corrispondente alla data
  df_domani.loc[df_domani['time'] == date, 'greed_and_fear'] = value

  # Carica il file JSON in un oggetto python
with open('data/difficulty.json', 'r') as file:
    data = json.load(file)

# Normalizza i dati del file JSON
df_difficulty = pd.json_normalize(data, 'difficulty', ['metric1', 'metric2'])

# Crea una lista di date partendo dal 23 dicembre 2021 fino al 22 dicembre 2022
dates = pd.date_range(start='2021-12-24', end='2022-12-22', freq='D')

# Crea una nuova colonna "date" nel tuo dataframe e assegna ad ogni riga una data
df_difficulty['date'] = dates

df_difficulty = df_difficulty.drop(['x', 'metric1', 'metric2'], axis=1)

df_difficulty.rename(columns={'date': 'time'}, inplace=True)

#trasforma df['time'] in datetime
df_difficulty['time'] = pd.to_datetime(df_difficulty['time'])

#seleziono solo la data di domani
#df_difficulty = df_difficulty.loc['2022-12-21':'2022-12-12']

df_difficulty = df_difficulty.iloc[::-1]

df_domani = pd.merge(df_domani, df_difficulty, on='time')

df_domani.rename(columns={'y': 'difficulty'}, inplace=True)

#istanzio l'auth
bearer_token = "AAAAAAAAAAAAAAAAAAAAAPgukgEAAAAADYQXq5tFdfahfgxE%2FSQqxcV4OFU%3DIETAxHE05aQI4saJCxFJMQiaurCSZgYMSHZ9kl7SMVNNxM5V4S"  # BEARER_TOKEN
auth = tweepy.Client(bearer_token)
api = tweepy.API(auth)

#scarico tweets di oggi

# Numero di giorni da scaricare
num_days = 1

# Numero di tweet da scaricare ogni giorno
num_tweets = 500

# Inizializza una lista che conterrà i tweet scaricati
tweet_list = []

# Inizializza la data di inizio del loop
start_date = today = datetime.today() - timedelta(days=2)
#start_date = datetime.now() - timedelta(days=30)

# Crea il loop
for i in range(num_days):
    # Aggiorna la data di fine
    end_date = start_date + timedelta(days=1)

    # Formatta le date come stringhe nel formato richiesto dall'API
    formatted_start_date = start_date.strftime("%Y-%m-%dT%H:%M:%S")
    formatted_end_date = end_date.strftime("%Y-%m-%dT%H:%M:%S")

    #hashtag
    query= "bitcoin"

    # Scarica i tweet
    response = auth.search_all_tweets(query, max_results=num_tweets, tweet_fields=["created_at", "lang"], start_time=start_date, end_time=end_date)
    tweets = response.data

    # Aggiungi i tweet alla lista
    for tweet in tweets:
        tweet_list.append([tweet.created_at, tweet.id, tweet.text, tweet.lang])

    # Aggiorna la data di inizio
    start_date = end_date
    time.sleep(5)

df_sentiment = pd.DataFrame(data=tweet_list, columns=['created_at', 'tweet_id', 'text', 'language'])  
df_sentiment.set_index("created_at", inplace = True)

df_sentiment = df_sentiment.iloc[::-1]

def preprocess_tweet(tweet):
    # Rimuove i caratteri di nuova riga e a capo
    tweet = tweet.replace("\n", " ").replace("\r", " ")
    
    # Rimuove i link
    tweet = re.sub(r"http\S+", "", tweet)
    
    # Rimuove gli hashtag
    tweet = re.sub(r"#\S+", "", tweet)
    
    # Rimuove le menzioni
    tweet = re.sub(r"@\S+", "", tweet)
    
    # Rimuove i caratteri di punteggiatura
    tweet = tweet.translate(str.maketrans("", "", string.punctuation))
    
    # Converti il testo in minuscolo
    tweet = tweet.lower()
    
    return tweet

# Crea una copia della colonna 'text' del dataframe
df_sentiment['text_clean'] = df_sentiment['text'].copy()

# Sostituisci i testi elaborati nella colonna 'text_clean' del dataframe
df_sentiment['text_clean'] = df_sentiment['text_clean'].apply(preprocess_tweet)

def get_sentiment(tweet_text):
    analysis = TextBlob(tweet_text)
    return analysis.sentiment.polarity

df_sentiment['sentiment'] = df_sentiment['text_clean'].apply(get_sentiment)

df_sentiment = df_sentiment.reset_index()
df_sentiment['created_at'] = pd.to_datetime(df_sentiment['created_at'])

# Crea una nuova colonna 'date' che estrae la data dalla colonna 'created_at'
df_sentiment['date'] = df_sentiment['created_at'].dt.date

# raggruppa i dati per data utilizzando la colonna 'data' come chiave di raggruppamento
grouped_data = df_sentiment.groupby('date')

# per ogni gruppo di dati, calcola la mediana del sentiment e aggiungi il valore alla lista medie
medie = []
for name, group in grouped_data:
    medie.append(mean(group['sentiment']))

# crea il dizionario con le date come chiavi e i valori medi come valori
dati = {key: value for key, value in zip(grouped_data.groups.keys(), medie)}

# crea il DataFrame utilizzando il dizionario appena creato
df_medie = pd.DataFrame.from_dict(dati, orient='index')

# rinomina la colonna del DataFrame con il nome desiderato
df_medie.rename(columns={0: 'sentiment_medio'}, inplace=True)

df_medie = df_medie.reset_index()

df_medie.rename(columns={'index': 'time'}, inplace=True)

df_medie = df_medie.iloc[::-1]

#trasformo la colonna created_at in datetime
df_medie['time'] = pd.to_datetime(df_medie['time'])

#trasforma df['time'] in datetime
df_domani['time'] = pd.to_datetime(df_domani['time'])

# Unisci i due dataframe utilizzando la colonna "time" come chiave di unione
df_domani = pd.merge(df_domani, df_medie, on='time')

pytrends = TrendReq(hl='en-US', tz=360)

# build payload

kw_list = ["bitcoin"] # list of keywords to get data 

pytrends.build_payload(kw_list, cat=0, timeframe='now 7-d')

#1 Interest over Time
data = pytrends.interest_over_time()

#riempio le date vuote con l'ultimo dato registrato
df_google = data.resample('D').ffill()

df_google = df_google.reset_index()

#df_google = df_google[6:7]

df_google.rename(columns={'date': 'time'}, inplace=True)

df_google.tail()

df_domani = pd.merge(df_domani, df_google, on='time')

df_domani = df_domani.drop(columns=['isPartial'])

In [37]:
df = df.reset_index()
df_domani = df_domani.reset_index()

# Unisci il dataframe di domani al dataframe storico
df = pd.concat([df_domani,df.loc[:]]).reset_index(drop=True)

df = df.set_index('time')


In [38]:
df.head()

,index,low,high,open,close,volume,greed_and_fear,difficulty,sentiment_medio,bitcoin,google_trend
time,,,,,,,,,,,
2022-12-22 00:00:00,0.0,16560.84,16868.82,16825.18,16815.51,21269.7673979,28,3.536407e+13,0.077123,75.0,NaN
2022-12-11,NaN,17073.19,17271.92,17128.1,17085.21,9948.85267,26,3.424433e+13,0.057855,NaN,30.0
2022-12-10,NaN,17093.42,17227.64,17130.49,17128.1,7860.586876,27,3.424433e+13,0.023664,NaN,28.0
2022-12-09,NaN,17060.69,17352.62,17226.03,17130.59,20976.636999,26,3.424433e+13,0.079547,NaN,28.0
2022-12-08,NaN,16738.0,17300.59,16839.76,17226.01,23533.234537,25,3.424433e+13,0.098894,NaN,28.0


In [39]:
domani = df.drop(columns=['close', 'index'])

In [44]:
tomorrow['low'] = tomorrow['low'].astype(float)
tomorrow['high'] = tomorrow['high'].astype(float)
tomorrow['volume'] = tomorrow['volume'].astype(float)
tomorrow['open'] = tomorrow['open'].astype(float)
tomorrow['greed_and_fear'] = tomorrow['greed_and_fear'].astype(int)

In [48]:
tomorrow['high'] = tomorrow['high'].replace(tomorrow['high'][0], np.nan)
tomorrow['low'] = tomorrow['low'].replace(tomorrow['low'][0], np.nan)
tomorrow['volume'] = tomorrow['volume'].replace(tomorrow['volume'][0], np.nan)
tomorrow.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 2022-12-22 00:00:00 to 2022-12-22 00:00:00
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   low              0 non-null      float64
 1   high             0 non-null      float64
 2   open             1 non-null      float64
 3   volume           0 non-null      float64
 4   greed_and_fear   1 non-null      int64  
 5   difficulty       1 non-null      float64
 6   sentiment_medio  1 non-null      float64
 7   bitcoin          1 non-null      float64
 8   google_trend     0 non-null      float64
dtypes: float64(8), int64(1)
memory usage: 80.0+ bytes


In [49]:
tomorrow.head()

,low,high,open,volume,greed_and_fear,difficulty,sentiment_medio,bitcoin,google_trend
time,,,,,,,,,
2022-12-22 00:00:00,NaN,NaN,16825.18,NaN,28,3.536407e+13,0.077123,75.0,NaN


In [50]:
# Usa il modello addestrato per fare previsioni sui dati di domani
next_day_predictions = model.predict(domani)

print(next_day_predictions)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type decimal.Decimal).